In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce

from datetime import datetime
from hts import HTSRegressor
from hts.utilities.load_data import load_hierarchical_sine_data
import hts.functions
import collections
from hts.hierarchy import HierarchyTree

## Hierarchical ARIMA by ags2 and ags5 

In [2]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide.csv', converters={'ags2': str, 'ags5': str}) 
df = df[['ags2','ags5','unemployment_rate', 'date']]

In [3]:
d = df.groupby('ags2')['ags5'].apply(lambda x: list(set(x))).to_dict()
d

{'01': ['01058',
  '01054',
  '01060',
  '01059',
  '01003',
  '01051',
  '01053',
  '01001',
  '01062',
  '01004',
  '01061',
  '01057',
  '01055',
  '01056',
  '01002'],
 '02': ['02000'],
 '03': ['03405',
  '03457',
  '03153',
  '03356',
  '03453',
  '03158',
  '03402',
  '03452',
  '03361',
  '03351',
  '03241',
  '03458',
  '03157',
  '03403',
  '03151',
  '03251',
  '03353',
  '03357',
  '03459',
  '03102',
  '03358',
  '03354',
  '03101',
  '03257',
  '03256',
  '03103',
  '03159',
  '03401',
  '03155',
  '03404',
  '03451',
  '03355',
  '03360',
  '03454',
  '03352',
  '03456',
  '03154',
  '03462',
  '03359',
  '03461',
  '03254',
  '03255',
  '03455',
  '03460',
  '03252'],
 '04': ['04012', '04011'],
 '05': ['05362',
  '05766',
  '05358',
  '05120',
  '05914',
  '05958',
  '05316',
  '05334',
  '05119',
  '05116',
  '05970',
  '05314',
  '05915',
  '05166',
  '05370',
  '05162',
  '05916',
  '05711',
  '05374',
  '05315',
  '05117',
  '05911',
  '05513',
  '05954',
  '05114',


In [4]:
level_names = ['ags2', 'ags5']
hierarchy = [['ags2'], ['ags5']]
df_wide, sum_mat, sum_mat_labels = hts.functions.get_hierarchichal_df(df,
                                                                    level_names=level_names,
                                                                    hierarchy=hierarchy,
                                                                    date_colname='date',
                                                                    val_colname='unemployment_rate')

In [5]:
df_wide['total'] = df_wide.sum(axis=1)
for key in d.keys():
    df_wide[key] = df_wide[[col for col in df_wide.columns if col.startswith(key)]].sum(axis=1)

In [6]:
df_wide.index = pd.to_datetime(df_wide.index)

In [7]:
d['total'] = list(d.keys())

In [8]:
ht = HierarchyTree.from_nodes(d, df_wide)
print(ht)

- total
   |- 01
   |  |- 01058
   |  |- 01054
   |  |- 01060
   |  |- 01059
   |  |- 01003
   |  |- 01051
   |  |- 01053
   |  |- 01001
   |  |- 01062
   |  |- 01004
   |  |- 01061
   |  |- 01057
   |  |- 01055
   |  |- 01056
   |  - 01002
   |- 02
   |  - 02000
   |- 03
   |  |- 03405
   |  |- 03457
   |  |- 03153
   |  |- 03356
   |  |- 03453
   |  |- 03158
   |  |- 03402
   |  |- 03452
   |  |- 03361
   |  |- 03351
   |  |- 03241
   |  |- 03458
   |  |- 03157
   |  |- 03403
   |  |- 03151
   |  |- 03251
   |  |- 03353
   |  |- 03357
   |  |- 03459
   |  |- 03102
   |  |- 03358
   |  |- 03354
   |  |- 03101
   |  |- 03257
   |  |- 03256
   |  |- 03103
   |  |- 03159
   |  |- 03401
   |  |- 03155
   |  |- 03404
   |  |- 03451
   |  |- 03355
   |  |- 03360
   |  |- 03454
   |  |- 03352
   |  |- 03456
   |  |- 03154
   |  |- 03462
   |  |- 03359
   |  |- 03461
   |  |- 03254
   |  |- 03255
   |  |- 03455
   |  |- 03460
   |  - 03252
   |- 04
   |  |- 04012
   |  - 04011
   |- 05
   |  

In [9]:
hp = HTSRegressor(transform=None, freq='3H')
hp.fit(df = df_wide, nodes = d)

Fitting models:   0%|          | 0/10 [00:01<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'fit'

In [10]:
hp = HTSRegressor(model='auto_arima', revision_method='OLS', n_jobs=0)
model = hp.fit(df_wide, d)

Fitting models: 100%|██████████| 418/418 [03:01<00:00,  2.30it/s]


AttributeError: 'HTSRegressor' object has no attribute 'predict_future'

In [12]:
predicted_autoarima = model.predict(steps_ahead=8)

Fitting models:   0%|          | 0/418 [00:00<?, ?it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1

Fitting models:  14%|█▎        | 57/418 [00:00<00:01, 283.95it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of

/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
Fitting models:  28%|██▊       | 116/418 [00:00<00:01, 281.69it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As o

Fitting models:  35%|███▍      | 146/418 [00:00<00:00, 287.94it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As o

Fitting models:  43%|████▎     | 178/418 [00:00<00:00, 296.68it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As o

...

Fitting models:  64%|██████▍   | 269/418 [00:00<00:00, 288.87it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As o

/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In fut

/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In fut

Fitting models:  91%|█████████▏| 382/418 [00:01<00:00, 248.61it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As o

/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In fut

In [13]:
predicted_autoarima

,total,01,02,03,04,05,06,07,08,09,...,16067,16052,16065,16056,16063,16071,16064,16066,16053,16068
2019-01-01,35.651222,0.275476,1.143006,-19.817314,-0.829980,2.437343,-0.784407,9.724750,-34.928325,-34.328181,...,-1.840074,11.267427,6.258911,2.841916,-1.840074,-3.898473,-1.840074,-1.840074,2.841916,-1.840074
2019-02-01,7886.019661,285.618274,70.505602,923.390095,106.192120,1063.444411,391.691624,568.896579,734.652157,1231.911638,...,-0.980581,18.766681,2.509032,17.120394,-2.180581,12.160376,1.119420,-1.980581,15.920394,-0.180581
2019-03-01,7779.108738,279.961368,69.376851,910.569803,105.327378,1049.654098,388.894234,560.934146,721.940298,1212.476691,...,-0.913921,18.487755,2.068789,16.746046,-2.213921,11.965678,1.044472,-2.113921,15.646046,-0.213921
2019-04-01,7475.596113,270.127997,66.861283,875.293154,103.203398,1031.444421,374.702887,542.291332,702.871934,1134.010033,...,-0.933901,17.983087,1.986527,16.188608,-2.004407,11.677585,0.965579,-2.004407,14.988608,-0.304407
2019-05-01,7151.817290,256.569076,63.896542,842.591228,100.467579,1024.166397,366.413436,526.393491,682.491953,1046.502584,...,-0.756232,17.235464,1.497702,15.406861,-2.077478,11.228718,0.680449,-2.177478,14.206861,-0.577478
2019-06-01,7083.181308,254.136773,63.982480,837.334598,100.855944,1029.993172,365.588118,524.974859,677.172609,1020.673884,...,-0.987199,17.237324,2.035239,15.232316,-1.995022,11.016155,0.880659,-2.195022,14.032316,-0.495022
2019-07-01,7002.055992,249.699581,62.903133,827.946879,99.828472,1022.444198,356.982809,523.643218,672.713687,1004.964271,...,-0.916846,16.801944,1.952500,15.220040,-2.010712,10.893742,0.695381,-2.210712,13.820040,-0.610712
2019-08-01,7191.901968,259.473891,64.984232,859.653546,101.434785,1058.825723,381.523692,553.712931,687.176448,1028.625822,...,-0.750031,17.264071,1.915542,15.585022,-2.114718,11.234113,0.942538,-2.114718,14.385022,-0.814718
2019-09-01,7356.273109,257.766327,65.832984,866.997020,103.353498,1063.635506,363.800232,531.309222,726.881272,1105.081230,...,-1.149436,17.261011,1.758552,15.585502,-2.016276,11.310023,0.753080,-2.116276,14.285502,-0.616276
2019-10-01,7061.525296,248.889848,63.316610,824.898443,99.873884,1000.462765,356.966668,520.469848,699.604713,1057.192492,...,-0.970983,16.717996,1.611853,14.968766,-1.916515,10.923946,0.639313,-1.816515,13.868766,-0.516515


In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(20, 10))

def plot_results(cols, axes, preds):
    axes = np.hstack(axes)
    for ax, col in zip(axes, cols):
        preds[col].plot(ax=ax, label="Predicted")
        df_wide[col].plot(ax=ax, label="Observed")

        ax.legend()
        ax.set_title(col)
        ax.set_xlabel("Date")
        ax.set_ylabel("N of Visitors")
    
plot_results(states, axes, predicted_autoarima)
plt.tight_layout()